In [99]:
import pandas as pd
import re

edges = pd.read_csv("service_matrix.csv").rename(columns={"Unnamed: 0" : "City", "Unnamed: 1" : "Label"}).set_index(["City", "Label"])
cities = list(edges.columns)

cityBasics = pd.read_csv("city-basics_iowa.csv").drop(["Unnamed: 0"], axis=1)

cityLat = dict()
cityLon = dict()

for city in cityBasics["City"].unique():
    cityLat[city] = cityBasics[cityBasics["City"] == city]["Lat"]
    cityLon[city] = cityBasics[cityBasics["City"] == city]["Lon"]

fail = []
    
for city in cities:
    if (city in cityLat.keys()):
        pass
    else:
        state = re.sub(".*?,\s+", "", city).strip()
        name = re.sub(",.*", "", city).strip()
        name = re.sub("\s+", "_", name)
        url = "http://citylatitudelongitude.com/" + state + "/" + name + ".htm"
        try:
            llDF = pd.read_html(url, header=0)[1]
            lat = float(re.sub("[A-Za-z\s]", "", llDF["Latitude"][0]))
            lon = float(re.sub("[A-Za-z\s]", "", llDF["Longitude"][0]))
            cityLat[city] = lat
            cityLon[city] = lon
        except:
            fail.append(city)

In [116]:
columnNames = edges.columns
newNames = dict()
for name in columnNames:
    newNames[name] = name.strip()
   

newedges = edges.rename(columns = newNames)

newedges = newedges.rename(columns={"Cherokee Iowa, IA": "Cherokee, IA"})
newedges["Decatur City, IA"] = newedges["Decatur, IA"]
newedges = newedges.drop(columns=["Old, IA", "Decatur, IA"], axis=1)
newfail = [x.strip() for x in fail]
newfail = newfail[2:][0:5] + newfail[2:][6:8]

In [161]:
finaledges = newedges
finaledges = finaledges.drop(columns=newfail, axis=1)
finaledges.to_csv("edges.csv")
cityLat = dict(zip(cityLat.keys(), [float(x) for x in cityLat.values()]))
cityLon = dict(zip(cityLon.keys(), [float(x) for x in cityLon.values()]))

In [178]:
llDF = pd.concat([pd.DataFrame.from_dict(cityLat, orient='index'), pd.DataFrame.from_dict(cityLon, orient='index')], axis=1)
llDF.columns = ["Lat", "Lon"]
llDF.to_csv("nodes.csv")

In [198]:
import geopy.distance

distances = finaledges

def getDistance(row):
    hLat = row['lat home']
    hLon = row['lon home']
    wLat = row['lat work']
    wLon = row['lon work']
    d = geopy.distance.vincenty((hLat, hLon), (wLat, wLon)).km
    return d

distances = distances.reset_index()
    
